In [5]:
import torch
import numpy as np
import sigpy as sp
from LinearMap import LinearMap

In [6]:
A = LinearMap([5,5], [5,5])

TypeError: 'module' object is not callable

In [7]:
LinearMap

<module 'LinearMap.LinearMap' from '/home/guanhua/mirtorch/LinearMap/LinearMap.py'>